```from langchain.chat_models import ChatOpenAI```

chat_models의 ChatOpenAI 모델을 살펴보면 model_name: str = "gpt-3.5-turbo" 이렇게 되어있는데, 이는 gpt 3.5 turbo 모델을 사용한다는 뜻입니다.
llms.openai는 model_name: str = "text-davinci-003" 이렇게 되어있습니다. 

openai 웹사이트에서(https://platform.openai.com/docs/models/gpt-3-5) 이 모델들의 차이점을 알아볼 수 있습니다. 

간단하게 둘을 비교해보자면 gpt 3.5 turbo는 text davinci 003 보다 chat에 특화되어 있습니다. 비용이 매우 저렴(1/10 수준)하기 때문입니다.  


In [1]:
# LLM 호출 

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic

In [2]:
!source /home/minyoungxi/MINYOUNGXI/fullstack-gpt/env/bin/activate

API가 통합되어 있어서 위의 langchain에서 사용하고 싶은 모델을 그대로 가져와서 사용할 수 있음. 

아래의 결과를 보면 대화형인 chatopenai 모델이 조금 더 대화 형식에 가까운 문장을 출력함

In [3]:
llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a,b

('\n\nThere are eight officially recognized planets in the Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate about the classification of Pluto, which was once considered the ninth planet but is now considered a dwarf planet by the International Astronomical Union.')

Chat model은 단지 질문을 받을 수 있을 뿐만 아니라 대화를 할 수도 있다는 뜻입니다. 

만약 model의 설정을 바꾸고 싶다면 model의 constructor(생성자)를 통해 할 수 있습니다.

ex1. max_tokens = model이 반환하는 결과의 최대 token을 정할 수 있음. 
ex2. temperature = model이 얼마나 창의적인지를 결정할 수 있음. 

In [4]:
chat = ChatOpenAI(
    temperature=0.1
)


predict messages는 위에서 본 것처럼 텍스트를 predict 하는 방법입니다. 질문을 하고 답변을 받는 방식이죠. 이번에는 messages들을 predict 할 것입니다. 

Humanmessage는 우리가 알고 있고, AIMessage는 AI가 보내는 메세지, SystemMessage는 우리가 LLM에 설정들을 제공하기 위한 Message입니다.

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

message = [
    SystemMessage(content = "You are a geography expert. And you only reply in Italian."),
    AIMessage(content = "Ciao ! mi chiamo Paolo"),
    HumanMessage(content='what is the distance between Mexico and Thailand. Also, what is your name?'),
] 

chat.predict_messages(message)

AIMessage(content='Ciao Paolo! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

### Prompt Templates 

- Prompt는 LLM과 의사소통할 수있는 유일한 방법임.

- more custome 해보자 ! 

- 아래의 코드를 통해 각 구성요소(components)를 잘 익혀보자 !! ( 나중에 어차피 한 라인으로 프롬프트와 템플릿을 사용할 수 있으니 )

In [8]:
from langchain.prompts import PromptTemplate , ChatPromptTemplate 

# PromptTemplate , ChatPromptTemplate  이 두 개의 프롬프트 템플릿은 다르다. 
# ChatPromptTemplate은 template을 message로부터 만듭니다. 반면 PromptTemplate은 String을 받아서 template을 만듭니다.

# example 

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)   

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [9]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao ! mi chiamo {name}"),
    (
        "human", 
        "what is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
] )

prompt = template.format_messages(
    language="Greek",
    name="MIN",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι MIN. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

### Output Parser and LCEL

In [10]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [18]:
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generationg machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else"),
    ('human', '{question}')
])

prompt = template.format_messages(
    max_items = 10, 
    question = "What are the colors?"  
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

### LCEL (LangChainExpressionLanguage)

- Lagchain의 전체적인 프로세스와 친해지자

In [19]:
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generationg machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else"),
    ('human', '{question}')
])

chain = template | chat | CommaOutputParser() # langchain의 핵심이다 ! 

chain.invoke({"max_items": 5, "question": "What are the poketmons?"})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

## Chaining Chains

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [15]:
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks = [StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients"),
    ('human', 'I want to cook {cuisine} food.'),
])

chef_chain = chef_prompt | chat

In [18]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

Great! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.

2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.

3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.

4. Add the chicken pieces to the pan and cook until they are lightly b

AIMessageChunk(content="Great! Butter Chicken is a delicious Indian dish, and I can help you make a vegetarian version of it. Instead of using chicken, we can substitute it with paneer, a type of Indian cheese. Here's the modified recipe:\n\nIngredients:\n- 500g paneer, cut into bite-sized pieces\n- 2 tablespoons butter\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (you can use a plant-based cream alternative)\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powde

## Recap 

- chef_chain의 출력값이 veg_chain의 입력값이 되도록.  

final_chain 의 runaable map이란 아래의 코드에서 "recipe": chef_chain 이 코드를 먼저 실행하고 응답결과를 다음으로 전달하도록 합니다. 

입력값이 무엇이든 상관이 없습니다. veg_chain에 recipe 값이 필요하기 때문에 이렇게 합니다. 

한 체인의 출력을 다음 체인의 입력으로 사용 (mapping과 유사함)

streaming=True 옵션은 LLM의 응답(response)이 생성되는대로 볼 수 있게 해줍니다. 전체 응답을 보기위해 작업이 끝날 때까지 기다릴 필요가 없습니다. 

callbacks는 볼 수 있는 문자가 생길 때마다 콘솔에 바로 print 해줍니다. 

```python
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})
```